<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import zipfile
import re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from gensim import corpora
from gensim.models.ldamulticore import LdaMulticore

import pyLDAvis.gensim
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings('ignore')

/opt/anaconda/anaconda3/envs/U4S1/lib/python3.7/site-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable


In [2]:
# creates gensim tokenizer
def tokenize(text):
    return [token for token in simple_preprocess(text) if token not in STOPWORDS]

In [3]:
# import amazon dataframe
amzn = pd.read_csv('data/amazon_data.zip')

# create column of tokenized data
amzn['tokens'] = amzn['reviews.text'].apply(lambda x: tokenize(x))

In [4]:
# creates dataframe with
df = amzn[['primaryCategories', 'brand', 'name', 'tokens', 'reviews.rating', ]].copy()
df.head()

,primaryCategories,brand,name,tokens,reviews.rating
0,Health & Beauty,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,"[order, item, bad, quality, missing, backup, s...",3
1,Health & Beauty,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,"[bulk, expensive, way, products, like]",4
2,Health & Beauty,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,"[duracell, price, happy]",5
3,Health & Beauty,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,"[work, brand, batteries, better, price]",5
4,Health & Beauty,Amazonbasics,AmazonBasics AAA Performance Alkaline Batterie...,"[batteries, long, lasting, price, great]",5


In [5]:
# creating corpora
id2word = corpora.Dictionary(df['tokens'])

# filter extreme values
id2word.filter_extremes(no_below=10, no_above=0.80)

# checks length of id2word
len(id2word)

2359

In [6]:
# creates corpus for each row
corpus = [id2word.doc2bow(text) for text in df['tokens']]

In [7]:
# # selects a product to look at

# product1 = df['name'].unique()[0]
# print(f'Product 1 is: {product1}')

# product2 = df['name'].unique()[3]
# print(f'Product 2 is: {product2}')

# product3 = df['name'].unique()[30]
# print(f'Product 2 is: {product3}')

In [8]:
# # creating list of tokens for each product
# p1t = df[df['name'] == product1].sum()
# p2t = df[df['name'] == product2].sum()
# p3t = df[df['name'] == product3].sum()

In [9]:
# setting up lda model
lda = LdaMulticore(corpus=corpus,
                   id2word=id2word,
                   num_topics=8,
                   passes=10,
                   workers=4
                  )

In [10]:
pyLDAvis.gensim.prepare(lda, corpus, id2word)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.247729 -0.055444       1        1  24.634520
2      0.226809 -0.092671       2        1  14.604375
5      0.002485 -0.158046       3        1  12.991312
4     -0.006242 -0.073119       4        1  10.657780
6     -0.083120  0.077620       5        1  10.564451
7      0.183873  0.091370       6        1   9.935810
0     -0.012683  0.201186       7        1   9.232080
1     -0.063392  0.009105       8        1   7.379674, topic_info=     Category         Freq         Term        Total  loglift  logprob
17    Default  7344.000000    batteries  7344.000000  30.0000  30.0000
20    Default  8897.000000        great  8897.000000  29.0000  29.0000
16    Default  4793.000000        price  4793.000000  28.0000  28.0000
1627  Default  3117.000000       kindle  3117.000000  27.0000  27.0000
33    Default  5596.000000         good  5596.000000  26.0000  26.0000
1496  Default  2149.000000        loves  2149.000000  25.0000  25.0000
1831  Default  6982.000000       tablet  6982.000000  24.0000  24.0000
679   Default  2981.000000         easy  2981.000000  23.0000  23.0000
185   Default  2534.000000         kids  2534.000000  22.0000  22.0000
101   Default  1811.000000         year  1811.000000  21.0000  21.0000
481   Default  2153.000000          old  2153.000000  20.0000  20.0000
1286  Default  1886.000000         apps  1886.000000  19.0000  19.0000
90    Default  4424.000000          use  4424.000000  18.0000  18.0000
27    Default  3126.000000       bought  3126.000000  17.0000  17.0000
19    Default  1248.000000        brand  1248.000000  16.0000  16.0000
22    Default  2456.000000         long  2456.000000  15.0000  15.0000
86    Default  3457.000000         love  3457.000000  14.0000  14.0000
188   Default  1219.000000          far  1219.000000  13.0000  13.0000
8     Default  2289.000000         work  2289.000000  12.0000  12.0000
594   Default  1721.000000        games  1721.000000  11.0000  11.0000
1633  Default  1515.000000        books  1515.000000  10.0000  10.0000
2     Default  2139.000000      battery  2139.000000   9.0000   9.0000
1370  Default  1634.000000       screen  1634.000000   8.0000   8.0000
706   Default  1252.000000         read  1252.000000   7.0000   7.0000
785   Default  1035.000000         gift  1035.000000   6.0000   6.0000
85    Default   723.000000        light   723.000000   5.0000   5.0000
832   Default  1223.000000      reading  1223.000000   4.0000   4.0000
1512  Default   848.000000         ipad   848.000000   3.0000   3.0000
1086  Default   930.000000     daughter   930.000000   2.0000   2.0000
73    Default   847.000000       brands   847.000000   1.0000   1.0000
...       ...          ...          ...          ...      ...      ...
160    Topic8   218.338409        times   431.933044   1.9242  -4.6161
521    Topic8    80.821793     included   130.564972   2.1268  -5.6099
39     Topic8   385.606445     purchase   950.409973   1.7044  -4.0473
135    Topic8   120.819969  replacement   229.058182   1.9668  -5.2078
15     Topic8   301.921204        happy   798.671326   1.6337  -4.2919
143    Topic8   119.333458      reviews   265.326813   1.8074  -5.2202
122    Topic8   351.500122    purchased  1363.930420   1.2505  -4.1399
1639   Topic8    73.602608         mini   136.203400   1.9910  -5.7034
1742   Topic8    88.314621      dropped   179.550201   1.8969  -5.5212
536    Topic8   143.404755       camera   398.133362   1.5853  -5.0364
27     Topic8   471.817780       bought  3126.812256   0.7153  -3.8455
76     Topic8   111.476715         hard   279.979187   1.6855  -5.2883
161    Topic8   214.060638           ve   907.567383   1.1619  -4.6358
212    Topic8   119.692657       worked   340.395996   1.5613  -5.2172
101    Topic8   274.498810         year  1811.080811   0.7197  -4.3872
213    Topic8   236.560211         best  1427.123779   0.8092  -4.5359
365    To

In [11]:
# shows topics with lecture regex
word_list = [re.findall(r'"([^"]*)"',t[1]) for t in lda.print_topics()]

# creates topic list
topics = []
for words in word_list:
    topic = ' '.join(words)
    topics.append(topic)   

In [12]:
lda_values = [lda[x] for x in corpus]

In [13]:
# topic_vals = []
# for x in range(len(lda_values)):
#     for i in range(9):
#         topic_vals.append(lda_values[x][i][1])

In [14]:
# lecture functrion for formating data to use in dataframe
def update(doc):
        d_dist = {k:0 for k in range(0,8)}
        for t in doc:
            d_dist[t[0]] = t[1]
        return d_dist
    
new_distro = [update(d) for d in lda_values]

In [15]:


DF = pd.DataFrame.from_records(new_distro)
DF.columns = topics
DF.head()

,kindle light read reading screen battery love like better new,far bought ipad purchase purchased tablet happy year working old,tablet kids loves old year great love bought games daughter,batteries good great price long work battery amazon brand buy,good tablet apps amazon screen use nice size perfect play,great tablet price good product works amazon value quality love,amazon tablet time like use charge work buy wifi alexa,easy use kindle love bought books read gift tablet reading
0,0.012507,0.012537,0.012504,0.713534,0.012535,0.012534,0.211338,0.012510
1,0.020857,0.020874,0.020839,0.854022,0.020855,0.020847,0.020854,0.020851
2,0.031261,0.208858,0.031258,0.603483,0.031258,0.031316,0.031260,0.031307
3,0.020849,0.020842,0.020834,0.854102,0.020841,0.020859,0.020838,0.020836
4,0.020837,0.020836,0.020842,0.854084,0.020836,0.020893,0.020836,0.020836


In [16]:
# what is the difference between from_recoreds and not using from records?
DF2 = pd.DataFrame(new_distro)
DF2.columns = topics
DF2.head()

,kindle light read reading screen battery love like better new,far bought ipad purchase purchased tablet happy year working old,tablet kids loves old year great love bought games daughter,batteries good great price long work battery amazon brand buy,good tablet apps amazon screen use nice size perfect play,great tablet price good product works amazon value quality love,amazon tablet time like use charge work buy wifi alexa,easy use kindle love bought books read gift tablet reading
0,0.012507,0.012537,0.012504,0.713534,0.012535,0.012534,0.211338,0.012510
1,0.020857,0.020874,0.020839,0.854022,0.020855,0.020847,0.020854,0.020851
2,0.031261,0.208858,0.031258,0.603483,0.031258,0.031316,0.031260,0.031307
3,0.020849,0.020842,0.020834,0.854102,0.020841,0.020859,0.020838,0.020836
4,0.020837,0.020836,0.020842,0.854084,0.020836,0.020893,0.020836,0.020836


In [17]:
# concats DF with category dataframe
finalDF = pd.concat([df['primaryCategories'], DF], axis=1)

In [18]:
# looks at topics used in different category reviews

finalDF.groupby('primaryCategories').mean()

,kindle light read reading screen battery love like better new,far bought ipad purchase purchased tablet happy year working old,tablet kids loves old year great love bought games daughter,batteries good great price long work battery amazon brand buy,good tablet apps amazon screen use nice size perfect play,great tablet price good product works amazon value quality love,amazon tablet time like use charge work buy wifi alexa,easy use kindle love bought books read gift tablet reading
primaryCategories,,,,,,,,
Animals & Pet Supplies,0.119633,0.145504,0.082952,0.141049,0.106739,0.245451,0.031277,0.119372
Electronics,0.092283,0.081155,0.210641,0.029242,0.116477,0.195677,0.089238,0.172800
"Electronics,Furniture",0.000000,0.000000,0.119464,0.000000,0.200271,0.000000,0.629416,0.000000
"Electronics,Media",0.563683,0.037766,0.023590,0.021713,0.042163,0.073561,0.076965,0.145292
Health & Beauty,0.041548,0.069638,0.033273,0.557638,0.048495,0.153950,0.052277,0.035608
Home & Garden,0.041691,0.041680,0.342622,0.157301,0.291653,0.041676,0.041686,0.041691
Office Supplies,0.068214,0.245575,0.028121,0.053607,0.307375,0.182194,0.028129,0.086785
"Office Supplies,Electronics",0.273397,0.065127,0.046532,0.035876,0.089003,0.149591,0.067755,0.261338
"Toys & Games,Electronics",0.038054,0.083030,0.516212,0.022656,0.094401,0.072140,0.078792,0.081422
